I'm not quite sure how to approach this problem: http://puzzlor.com/2014-12_Electrifying.html so I'm going to work through this http://isaacslavitt.com/2015/02/26/combinatorial-optimization/ solution set to understand where I can apply these tools to other puzzlor problems

In [1]:
%matplotlib inline
from matplotlib import pyplot as plt
import numpy as np
np.set_printoptions(precision=2)

In [ ]:
neighborhoods = ['A6', 'B2', 'B4', 'B5', 'B7', 'C5', 'C10', 'D9',
                 'E2', 'E6', 'E8', 'F3', 'F5', 'G8', 'G9', 'H3',
                 'H5', 'H7', 'H8', 'J4']
array_to_alphanum = lambda(i,j): '{}{}'.format('JIHGFEDCBA'[i],range(1,11)[j])
alphanum_to_array = lambda n: 